# The following code is responsible for converting MIDI files into text notation.

The whole code is based on the following model: https://keunwoochoi.wordpress.com/2016/02/23/lstmetallica/ 

# I found a way to use the original python library!

The original code was developed using Python2 and the Python-midi library, which is not available for Python3.

Basically i found a Python3 compatible version of the original library.


Reference here: https://github.com/jameswenzel/mydy/blob/master/src/Containers.py
https://github.com/jameswenzel/Fractal-Midi/blob/master/script.py
https://github.com/vishnubob/python-midi

## Opening MIDI file
Basically the next cell opens and reads a dummy MIDI file written by me.

I'm also setting the  ```resolution ``` parameter to 480. This parameter is equivalent to **PPQ** in MIDI files.

**PPQ** (*Pulse per Quarter Note*) is a fixed value which sets the number of pulses contained in a quarter note, it's like a "sampling" frequency.

Each quarter note (no matter what is the original speed of the song) will contain 480 pulses. Then these pulses are converted into actual playback using the **Tempo** information of the MIDI file (obviously a quarter note at 100 BPM is slower than a quarter note at 130 BPM)

In [20]:
from mydy import Events, FileIO, Containers, Constants


In [28]:
from mydy import Events, FileIO, Containers, Constants
test=FileIO.read_midifile('bellofigo forse meglio.mid') #returns a Pattern with the MIDI file information (resolution ecc...), based on documentation https://github.com/jameswenzel/mydy/blob/master/src/FileIO.py

test.resolution=480 #qui sto settando quanti tick ho in una quarter note. Quindi ogni quarter note avra 480 ticks.
print(test) #seems that changing the BPM doesn't influence the ticks.
#Resolution is the same as PPQ



mydy.Pattern(format=1, resolution=480, tracks=\
[mydy.Track(relative: True\
  [mydy.SetTempoEvent(tick=0.0, data=[10, 197, 90]),
   mydy.TimeSignatureEvent(tick=0.0, data=[4, 2, 24, 8]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[36, 100]),
   mydy.NoteOffEvent(tick=0.0, channel=0, data=[36, 64]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[42, 84]),
   mydy.NoteOffEvent(tick=30.00091555528428, channel=0, data=[42, 64]),
   mydy.NoteOnEvent(tick=210.00640888698996, channel=0, data=[42, 72]),
   mydy.NoteOffEvent(tick=30.00091555528428, channel=0, data=[42, 64]),
   mydy.NoteOnEvent(tick=210.00640888698996, channel=0, data=[38, 100]),
   mydy.NoteOffEvent(tick=0.0, channel=0, data=[38, 64]),
   mydy.NoteOnEvent(tick=120.00366222113712, channel=0, data=[36, 95]),
   mydy.NoteOffEvent(tick=0.0, channel=0, data=[36, 64]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[42, 74]),
   mydy.NoteOffEvent(tick=30.00091555528428, channel=0, data=[42, 64]),
   mydy.NoteOnEvent(tick=10.005188

## Reading notes

The following cell access the loaded MIDI file and reads the  ```Track```. Using the  ```mydy``` library, MIDI files are structured in the following way:

 ```Pattern -> Track -> MIDI_Events ```
 
 This means that, whenever I open a MIDI file, i will get a  ```Pattern ```, which contains  ```Track ```, which contains ```MIDI_Events ```.
 
 This scructure is used because a single MIDI file can contain multiple instruments (guitar, drums, bass ecc...), so each  ```Track ``` corresponds to an instrument, and the  ```MIDI_Events ``` are the note played by the single instrument.
 
 In our case we can assume to be working with single  ```Track ``` MIDI files (we're interested only in drums).
 
 **NB**: i'm using  ```track.make_ticks_abs()``` to convert the time (expressed in ticks/PPQ) from a relative value into an absolute one.<br>The standard representation of MIDI files represents note based on the time elapsed by the previous note. With this command i'm converting the time from relative to absolute (each note is represented with the time elapsed by the beginning of the song) 

In [51]:

track = test[1] #selecting the track (since it's only one, it will be always at index 0)
track_abs = track.make_ticks_abs()# Converting time from relative to an absolute measure


filtered_list=track_abs.filter(lambda e: isinstance(e, Events.NoteOnEvent))# Selects only Note_On events, i'm discarding the note off

for element in filtered_list:
    
    print(element.tick)

#Just a bunch of test printings 
#print(type(filtered_list))
#print(filtered_list)
#print(len(filtered_list))
#print(filtered_list[2].data[1])
#print(test.resolution)


0.0
600.0
720.0
1200.0
1560.0
1920.0
2520.0
2640.0
3120.0
3480.0
3840.0
4440.0
4560.0
5040.0
5400.0
5760.0
6360.0
6480.0
6960.0
7320.0


# Automatic MIDI extraction from multiple files

In [21]:
#function to approximate te length to the next bar https://stackoverflow.com/questions/3407012/c-rounding-up-to-the-nearest-multiple-of-a-number
def roundup(numToRound, multiple):
    if multiple == 0:
        return numToRound
    remainder = numToRound % multiple
    if remainder==0:
        return numToRound
    return numToRound + multiple - remainder

    

In [22]:
import os
#Fix here the problem with the tick. Now i'm just adding the last tick to the next track. This makes the new track starting right from there
#instead of waiting for the new BAR. I have to quantize it and make it start from the new BAR 
pattern = Containers.Pattern(fmt=0)
max_time=0;
approx_bar = 480*4
pattern.resolution= 480
for filename in os.listdir("Dataset NEW"):
    print(os.path.join("Dataset NEW",filename))
    path= os.path.join("Dataset NEW",filename)
    
    test=FileIO.read_midifile(path)
    test.resolution = 480
    test=test[0].make_ticks_abs()
    test=test.filter(lambda e: isinstance(e, Events.NoteOnEvent))# Selects only Note_On events, i'm discarding the note off
    print("printo la singola track")
    print(test)
    for note in test: #probabile bug qui! il valore massimo di tick con i dati attuali dovrebbe essere 61320! BUG QUI! SICURO!
        note.tick = note.tick + max_time
    print('max time before')
    print(max_time)
    max_time = test[-1].tick
    #insert here round up on max_time
    max_time=roundup(max_time, approx_bar)
    print('max time after')
    print(max_time)
    
    pattern.append(test)
    
print(pattern)



Dataset NEW\acjbwmxu.mid
printo la singola track
mydy.Track(relative: False\
  [mydy.NoteOnEvent(tick=0.0, channel=0, data=[36, 100]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[42, 103]),
   mydy.NoteOnEvent(tick=480.0, channel=0, data=[42, 86]),
   mydy.NoteOnEvent(tick=960.0, channel=0, data=[38, 100]),
   mydy.NoteOnEvent(tick=960.0, channel=0, data=[42, 89]),
   mydy.NoteOnEvent(tick=1440.0, channel=0, data=[42, 98]),
   mydy.NoteOnEvent(tick=1920.0, channel=0, data=[42, 97]),
   mydy.NoteOnEvent(tick=2400.0, channel=0, data=[36, 100]),
   mydy.NoteOnEvent(tick=2400.0, channel=0, data=[42, 105]),
   mydy.NoteOnEvent(tick=2880.0, channel=0, data=[38, 100]),
   mydy.NoteOnEvent(tick=2880.0, channel=0, data=[42, 99]),
   mydy.NoteOnEvent(tick=3360.0146488845485, channel=0, data=[42, 83]),
   mydy.NoteOnEvent(tick=3480.018311105686, channel=0, data=[42, 101]),
   mydy.NoteOnEvent(tick=3600.021973326823, channel=0, data=[42, 86]),
   mydy.NoteOnEvent(tick=3720.0256355479605, channe

printo la singola track
mydy.Track(relative: False\
  [mydy.NoteOnEvent(tick=0.0, channel=0, data=[36, 100]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[42, 108]),
   mydy.NoteOnEvent(tick=480.0, channel=0, data=[42, 96]),
   mydy.NoteOnEvent(tick=960.0, channel=0, data=[38, 100]),
   mydy.NoteOnEvent(tick=960.0, channel=0, data=[42, 98]),
   mydy.NoteOnEvent(tick=1440.0, channel=0, data=[42, 104]),
   mydy.NoteOnEvent(tick=1920.0, channel=0, data=[42, 103]),
   mydy.NoteOnEvent(tick=2400.0, channel=0, data=[36, 100]),
   mydy.NoteOnEvent(tick=2400.0, channel=0, data=[42, 108]),
   mydy.NoteOnEvent(tick=2880.0, channel=0, data=[36, 100]),
   mydy.NoteOnEvent(tick=2880.0, channel=0, data=[38, 100]),
   mydy.NoteOnEvent(tick=2880.0, channel=0, data=[42, 104]),
   mydy.NoteOnEvent(tick=3360.0, channel=0, data=[42, 94]),
   mydy.NoteOnEvent(tick=3840.0, channel=0, data=[42, 106]),
   mydy.NoteOnEvent(tick=4320.0, channel=0, data=[42, 96]),
   mydy.NoteOnEvent(tick=4800.0, channel=0, da

mydy.Pattern(format=0, resolution=480, tracks=\
[mydy.Track(relative: False\
  [mydy.NoteOnEvent(tick=0.0, channel=0, data=[36, 100]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[42, 103]),
   mydy.NoteOnEvent(tick=480.0, channel=0, data=[42, 86]),
   mydy.NoteOnEvent(tick=960.0, channel=0, data=[38, 100]),
   mydy.NoteOnEvent(tick=960.0, channel=0, data=[42, 89]),
   mydy.NoteOnEvent(tick=1440.0, channel=0, data=[42, 98]),
   mydy.NoteOnEvent(tick=1920.0, channel=0, data=[42, 97]),
   mydy.NoteOnEvent(tick=2400.0, channel=0, data=[36, 100]),
   mydy.NoteOnEvent(tick=2400.0, channel=0, data=[42, 105]),
   mydy.NoteOnEvent(tick=2880.0, channel=0, data=[38, 100]),
   mydy.NoteOnEvent(tick=2880.0, channel=0, data=[42, 99]),
   mydy.NoteOnEvent(tick=3360.0146488845485, channel=0, data=[42, 83]),
   mydy.NoteOnEvent(tick=3480.018311105686, channel=0, data=[42, 101]),
   mydy.NoteOnEvent(tick=3600.021973326823, channel=0, data=[42, 86]),
   mydy.NoteOnEvent(tick=3720.0256355479605, channe

In [23]:
filtered_list = []

for track in pattern:
    print("printo una track")
    print(track)
    filtered_list.append(track)
len(filtered_list)

printo una track
mydy.Track(relative: False\
  [mydy.NoteOnEvent(tick=0.0, channel=0, data=[36, 100]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[42, 103]),
   mydy.NoteOnEvent(tick=480.0, channel=0, data=[42, 86]),
   mydy.NoteOnEvent(tick=960.0, channel=0, data=[38, 100]),
   mydy.NoteOnEvent(tick=960.0, channel=0, data=[42, 89]),
   mydy.NoteOnEvent(tick=1440.0, channel=0, data=[42, 98]),
   mydy.NoteOnEvent(tick=1920.0, channel=0, data=[42, 97]),
   mydy.NoteOnEvent(tick=2400.0, channel=0, data=[36, 100]),
   mydy.NoteOnEvent(tick=2400.0, channel=0, data=[42, 105]),
   mydy.NoteOnEvent(tick=2880.0, channel=0, data=[38, 100]),
   mydy.NoteOnEvent(tick=2880.0, channel=0, data=[42, 99]),
   mydy.NoteOnEvent(tick=3360.0146488845485, channel=0, data=[42, 83]),
   mydy.NoteOnEvent(tick=3480.018311105686, channel=0, data=[42, 101]),
   mydy.NoteOnEvent(tick=3600.021973326823, channel=0, data=[42, 86]),
   mydy.NoteOnEvent(tick=3720.0256355479605, channel=0, data=[42, 90]),
   mydy.Not

44

In [24]:
for element in filtered_list:
    for note in element:
        print(note.tick)

0.0
0.0
480.0
960.0
960.0
1440.0
1920.0
2400.0
2400.0
2880.0
2880.0
3360.0146488845485
3480.018311105686
3600.021973326823
3720.0256355479605
3840.029297769098
4320.043946653646
4800.043946653646
4800.043946653646
5280.043946653646
5280.043946653646
5760.043946653646
6240.043946653646
6240.043946653646
6410.044251838742
6600.054933317058
6720.058595538196
6720.058595538196
7020.067751091039
7200.073244422745
7320.0769066438825
7440.08056886502
7680.0878933072945
7680.0878933072945
8160.0878933072945
8640.087893307295
8640.087893307295
9120.087893307295
9600.087893307295
10080.087893307295
10080.087893307295
10560.087893307295
10560.087893307295
11040.087893307295
11160.091555528432
11280.09521774957
11400.098879970707
11520.102542191844
12000.102542191844
12480.102542191844
12480.102542191844
12960.102542191844
12960.102542191844
13080.106204412981
13200.109866634119
13320.113528855256
13440.117191076393
13920.117191076393
13920.117191076393
14090.117496261488
14280.128177739805
14400.

527040.0439466536
527040.0439466536
527520.0439466536
528000.0439466536
528480.0439466536
528480.0439466536
528650.0442518387
528840.054933317
528960.0585955381
528960.0585955381
529260.0677510911
529560.0769066439
529920.0878933073
529920.0878933073
530400.0878933073
530880.0878933073
530880.0878933073
531360.0878933073
531840.0878933073
532160.0927762688
532320.0878933073
532480.0976592303
532800.1025421919
532800.1025421919
532800.1025421919
533280.1025421919
533400.106204413
533520.1098666341
533640.1135288553
533760.1171910764
534240.1171910764
534720.1171910764
534720.1171910764
534720.1171910764
535200.1171910764
535320.1208532975
535440.1245155187
535560.1281777398
535680.1318399609
536160.1318399609
536330.132145146
536520.1428266243
536640.1464888455
536640.1464888455
537000.1574755089
537240.1647999511
537600.0
537600.0
538080.0
538560.0
538560.0
538560.0
539040.0
539520.0
540000.0
540000.0
540480.0
540480.0
540480.0
540960.0
541440.0
541920.0
542400.0
542400.0
542400.0
5428

## Creating couples (Pitch, time)
So basically now i'm processing the previous raw data (as you can see from the printings, it's quite a mess) in order to obtain couples of ```(pitch, time)``` for each note.


In [25]:
notes = []
couple = ()
for element in filtered_list:
    for note in element:
            ### test printings
        print(note.tick)
        print(note.data[0])
    
    
        notes.append(tuple((note.data[0],note.tick))) #storing couples of (note_pitch,tick_time)
        



0.0
36
0.0
42
480.0
42
960.0
38
960.0
42
1440.0
42
1920.0
42
2400.0
36
2400.0
42
2880.0
38
2880.0
42
3360.0146488845485
42
3480.018311105686
42
3600.021973326823
42
3720.0256355479605
42
3840.029297769098
42
4320.043946653646
42
4800.043946653646
38
4800.043946653646
42
5280.043946653646
36
5280.043946653646
42
5760.043946653646
42
6240.043946653646
36
6240.043946653646
42
6410.044251838742
42
6600.054933317058
42
6720.058595538196
38
6720.058595538196
42
7020.067751091039
42
7200.073244422745
36
7320.0769066438825
42
7440.08056886502
36
7680.0878933072945
36
7680.0878933072945
42
8160.0878933072945
42
8640.087893307295
38
8640.087893307295
42
9120.087893307295
42
9600.087893307295
42
10080.087893307295
36
10080.087893307295
42
10560.087893307295
38
10560.087893307295
42
11040.087893307295
42
11160.091555528432
42
11280.09521774957
42
11400.098879970707
42
11520.102542191844
42
12000.102542191844
42
12480.102542191844
38
12480.102542191844
42
12960.102542191844
36
12960.102542191844
42

42
251040.0585955382
36
251040.0585955382
42
251280.06591998046
38
251520.07324442273
42
252000.07324442273
36
252000.07324442273
42
252170.07354960783
42
252360.08423108616
42
252480.0878933073
38
252480.0878933073
42
252780.09704886013
42
252960.10254219183
36
253080.106204413
42
253200.10986663413
36
253440.1171910764
36
253440.1171910764
42
253920.1171910764
42
254400.1171910764
38
254400.1171910764
42
254880.1171910764
42
255120.12451551866
38
255360.13183996093
42
255840.13183996093
36
255840.13183996093
42
256320.13183996093
38
256320.13183996093
42
256800.13183996093
42
256920.13550218206
42
257040.13916440323
42
257160.14282662436
42
257280.1464888455
42
257760.1464888455
42
258240.1464888455
38
258240.1464888455
42
258720.1464888455
36
258720.1464888455
42
258840.15015106663
42
258960.15381328776
38
258960.15381328776
42
259080.1574755089
42
259200.16113773003
42
259680.16113773003
36
259680.16113773003
42
259850.16144291512
42
260040.17212439346
42
260150.170598468
38
260160

42
429660.06042664876
42
429720.06225775933
42
429780.0640888699
42
429840.06591998046
42
429900.06775109103
42
430080.0
36
430080.0
42
430560.0
42
431040.0
38
431040.0
42
431520.0
36
431520.0
42
431760.00732444227
38
432000.01464888453
42
432480.01464888453
42
432960.01464888453
38
432960.01464888453
42
433200.0219733268
36
433440.02929776907
42
433560.03295999026
42
433680.0366222114
42
433800.0402844325
42
433920.04394665366
36
433920.04394665366
42
434400.0585955382
42
434880.0585955382
38
434880.0585955382
42
435360.0585955382
36
435360.0585955382
42
435600.06591998046
38
435840.07324442273
42
436320.07324442273
42
436490.0735496078
42
436680.08423108613
42
436800.08789330727
36
436800.08789330727
38
436800.08789330727
42
437100.09704886016
42
437400.106204413
42
437760.1171910764
36
437760.1171910764
42
438240.1171910764
42
438720.1171910764
38
438720.1171910764
42
439200.1171910764
36
439200.1171910764
42
439440.12451551866
38
439680.13183996093
42
440160.13183996093
42
440640.1

42
634560.0585955381
38
634560.0585955381
42
635040.0585955381
36
635040.0585955381
42
635280.0659199805
38
635520.0732444228
42
636000.0732444228
42
636170.0735496079
42
636360.0842310862
42
636480.0878933073
36
636480.0878933073
38
636480.0878933073
42
636780.0970488602
42
637080.106204413
42
637440.1171910764
36
637440.1171910764
42
637920.1171910764
42
638400.1171910764
38
638400.1171910764
42
638880.1171910764
36
638880.1171910764
42
639120.1245155187
38
639360.1318399609
42
639840.1318399609
42
640320.1318399609
38
640320.1318399609
42
640560.1391644032
36
640800.1464888455
42
640920.1501510666
42
641040.1538132877
42
641160.1574755089
42
641280.16113773
36
641280.16113773
42
641760.16113773
42
642240.16113773
38
642240.16113773
42
642720.16113773
36
642720.16113773
42
642840.1647999511
42
642960.1684621723
38
642960.1684621723
42
643080.1721243934
42
643200.1757866145
42
643680.1757866145
42
643850.1760917996
42
644040.186773278
42
644150.1852473526
38
644160.1904354992
36
64416

In [52]:
notes = []
couple = ()

for element in filtered_list:
    
    ### test printings
    print(element.tick)
    print(element.data[0])
    ###
    
    notes.append(tuple((element.data[0],element.tick))) #storing couples of (note_pitch,tick_time)


print('printing tuples')
print(notes)


0.0
60
600.0
60
720.0
60
1200.0
60
1560.0
60
1920.0
60
2520.0
60
2640.0
60
3120.0
60
3480.0
60
3840.0
60
4440.0
60
4560.0
60
5040.0
60
5400.0
60
5760.0
60
6360.0
60
6480.0
60
6960.0
60
7320.0
60
printing tuples
[(60, 0.0), (60, 600.0), (60, 720.0), (60, 1200.0), (60, 1560.0), (60, 1920.0), (60, 2520.0), (60, 2640.0), (60, 3120.0), (60, 3480.0), (60, 3840.0), (60, 4440.0), (60, 4560.0), (60, 5040.0), (60, 5400.0), (60, 5760.0), (60, 6360.0), (60, 6480.0), (60, 6960.0), (60, 7320.0)]


In [7]:
#RANDOM TEST ON THE LIBRARY, USELESS
test= Constants.C_3
print(test)

36


## Trouble and make it double
Now we're going into the tough part.

The following cell is an *utility* in order to convert our MIDI into a text file (that we will use to traing our network, read the reference model here: https://keunwoochoi.wordpress.com/2016/02/23/lstmetallica/)

Basically it creates 2 classes: ```Note``` and ```Note_List```.

**1)** *Note*:    Creates a simple ```Note``` object composed by ```pitch``` ```c_tick``` and ```idx```. ```pitch``` and ```c_tick``` are the already mentioned pitch and time, while ```idx``` is a variable that counts the index of my note on a 16-th note reference. <br>**For example**: if i have 2 whole notes, the second note will have ```idx=16```.

**2)** *Note_list*:  Creates an empty list where we will add our *Note* objects. It also contains supports methods used to create and manage this list.

**List of methods**: I'll add some comments to the code and make them more readable.


In [26]:
from mydy import Events, FileIO, Containers
import pdb

#Function responsible for converting midi notes into text. Since i have to train my network over the structure i decided
#which is 0b0000000 for no note, 0b01000000 for kick ecc... i need to convert midi notes into this format.

#The original script used for midi-text translation has been lost, must be re-implemented again
PPQ = 480 # Pulse per quater note. Used in sequencers. Standard value
event_per_bar = 32 # to quantise.
min_ppq = PPQ / (event_per_bar/4)

# ignore: 39 hand clap, 54 tambourine, 56 Cowbell, 58 Vibraslap, 60-81

#the dictionary below maps values to other ones. Reduced the size of the used notes. For example
#if i have an eletric snare or a stick snare, i just map both of them into a standard snare

drum_conversion = {35:36, # acoustic bass drum -> bass drum (36)
                    37:38, 40:38, # 37:side stick, 38: acou snare, 40: electric snare
                    43:41, # 41 low floor tom, 43 ghigh floor tom
                    47:45, # 45 low tom, 47 low-mid tom
                    50:48, # 50 high tom, 48 hi mid tom
                    44:42, # 42 closed HH, 44 pedal HH
                    57:49, # 57 Crash 2, 49 Crash 1
                    59:51, 53:51, 55:51, # 59 Ride 2, 51 Ride 1, 53 Ride bell, 55 Splash
                    52:49 # 52: China cymbal
                    }

#Used in the code to map elements, everything that has not one of the following number is discarded.
#Basically i'm ignoring notes that are not in my dataset (for examle i'll ignore shakers ecc...)
                # k, sn,cHH,oHH,LFtom,ltm,htm,Rde,Crash
allowed_pitch = [36, 38, 42, 46, 41, 45, 48, 51, 49] # 46: open HH
cymbals_pitch = [49, 51] # crash, ride
cymbals_pitch = [] # crash, ride
# pitch_to_midipitch = {36:midi.C_2, # kick # for general MIDI Drum map
# 						38:midi.D_2, # Snare
# 						39:midi.Eb_2, # hand clap (it's alive by mistake..)
# 						41:midi.F_2, # Low floor tom
# 						42:midi.Gb_2, # Close HH
# 						45:midi.A_2, # Low tom
# 						46:midi.Bb_2, # Open HH
# 						48:midi.C_3,  # Hi Mid Tom
# 						49:midi.Db_3, # Crash
# 						51:midi.Eb_3 # Ride
# 						}

#mapping midi values into notes
pitch_to_midipitch = {36:Constants.C_3,  # for logic 'SoCal' drum mapping
                        38:Constants.D_3, 
                        39:Constants.Eb_3,
                        41:Constants.F_3,
                        42:Constants.Gb_3,
                        45:Constants.A_3,
                        46:Constants.Bb_3,
                        48:Constants.C_4,
                        49:Constants.Db_4,
                        51:Constants.Eb_4
                        }
#la singola nota è un elemento composto da pitch (numerico, pitch midi) e tick (modo per tenere il tempo in midi)
class Note:
    def __init__(self, pitch, c_tick):
        self.pitch = pitch
        self.c_tick = c_tick # cumulated_tick of a midi note

    def add_index(self, idx):
        '''index --> 16-th note-based index starts from 0'''
        self.idx = idx

class Note_List():
    def __init__(self):
        ''''''
        self.notes = []
        self.quantised = False
        self.max_idx = None

    def add_note(self, note):
        '''note: instance of Note class'''
        self.notes.append(note)

    def quantise(self, minimum_ppq):
        '''
        e.g. if minimum_ppq=120, quantise by 16-th note.
        
        '''
        if not self.quantised:
            for note in self.notes:
                note.c_tick = ((note.c_tick+minimum_ppq/2)//minimum_ppq)* minimum_ppq # quantise
                #here the index is calculated. The index is an absolute index over the 16th notes.
                #for example an index of value 34, means that my current note appears after 34 chromes
                #it's simply calculated by dividing the cumulated tick of the note by the ticks contained in a 16th note
                note.add_index(note.c_tick/minimum_ppq)
            #NB: THE QUANTIZATION FUNCTION ITERATES OVER ALL THE NOTES. So first i add all the notes, then i iterate and quantize

            #Does this automatically reference to the last item added?
            #YES. The counter note will store the last element of the iteration. So basically here i'm assigning as max index the index of the last added note
            self.max_idx = note.idx

            #Here checks if if my ending is a full musical bar. For example, if my file ends with a single kick, i'll add that note.
            #but that kick will (probably) be at the beginning of the last musical bar. So i have to "pad" until the end.
            #It's like adding a pause on my piece, so i have all complete bars and no trucated ones at the end
            if (self.max_idx + 1) % event_per_bar != 0:
                self.max_idx += event_per_bar - ((self.max_idx + 1) % event_per_bar) # make sure it has a FULL bar at the end.
            self.quantised = True

        return

    def simplify_drums(self):
        ''' use only allowed pitch - and converted not allowed pitch to the similar in a sense of drums!
        '''
        #Here forces conversion into the pitches in drum_conversion
        for note in self.notes:
            if note.pitch in drum_conversion: # ignore those not included in the key
                note.pitch = drum_conversion[note.pitch]
        #https://stackoverflow.com/questions/30670310/what-do-brackets-in-a-for-loop-in-python-mean
        #The following one is a list comprehension. Basically generates a new list from an existing one using a given condition on the elements
        self.notes = [note for note in self.notes if note.pitch in allowed_pitch]	

        return

    def return_as_text(self):
        ''''''
        length = int(self.max_idx + 1) # of events in the track.
        #print(type(length))
        event_track = []
        #Thw following cycle create a 9 by N matrix. I append N times a vector of nine zeros.
        #This means that i create N notes, and then i initialize them with all zeros (9 zeros, since a note is represented by a 9 element binary number)

        for note_idx in range(length):  #sostituire xrange con range in Python3
            event_track.append(['0']*len(allowed_pitch))

        num_bars = length/event_per_bar# + ceil(len(event_texts_temp) % _event_per_bar)

        for note in self.notes:
            pitch_here = note.pitch
            #The following line returns the index of the passed pitch. Basically given an input generic pitch
            #it returns the associated pitch in my vocabolary (computes the actual mapping from the whole
            #vocabolary of notes into my reduced one)
            note_add_pitch_index = allowed_pitch.index(pitch_here) # 0-8
            #print(type(note.idx))  
            #print(type(note_add_pitch_index))
            event_track[int(note.idx)][note_add_pitch_index] = '1'
            # print note.idx, note.c_tick, note_add_pitch_index, ''.join(event_track[note.idx])
            # pdb.set_trace()

        event_text_temp = ['0b'+''.join(e) for e in event_track] # encoding to binary

        event_text = []
        # event_text.append('SONG_BEGIN')
        # event_text.append('BAR')
        print(num_bars)
        print(type(num_bars))        
        for bar_idx in range(int(num_bars)):
            event_from = bar_idx * event_per_bar
            event_to = event_from + event_per_bar
            event_text = event_text + event_text_temp[event_from:event_to]
            event_text.append('BAR')

        # event_text.append('SONG_END')

        return ' '.join(event_text)


## Creating Note_List
As simple as that, i'm taking my starting list of couples ```(pitch,time)``` into ```Note```  objects. 

Why? Because you should recycle also code, not only plastic. LUL



In [27]:

##NB: AGGIUNGERE GLI idx ALLE SINGLE NOTES! 
note_list = Note_List()
for note in notes:
    pitch = note[0]
    tick = note[1]
    idx = int(tick / min_ppq)
    new_note = Note(pitch,tick)
    new_note.add_index(idx)
    note_list.add_note(new_note)


In [28]:
for note in note_list.notes:
    print(note.idx)

0
0
8
16
16
24
32
40
40
48
48
56
58
60
62
64
72
80
80
88
88
96
104
104
106
110
112
112
117
120
122
124
128
128
136
144
144
152
160
168
168
176
176
184
186
188
190
192
200
208
208
216
216
218
220
222
224
232
232
234
238
240
240
246
250
255
255
263
271
271
280
280
288
296
304
304
304
312
314
316
318
320
328
328
336
336
344
352
360
362
366
368
368
368
373
378
384
384
392
400
400
408
408
416
424
432
432
432
440
442
444
446
448
456
456
464
464
472
474
476
478
480
488
490
494
496
496
502
506
512
512
520
528
528
536
536
544
552
560
560
564
568
576
576
584
592
592
600
600
602
604
606
608
616
620
624
624
624
629
634
640
640
648
656
656
664
664
672
680
688
688
692
696
704
704
712
720
720
728
728
730
732
734
736
744
748
752
752
752
753
754
755
756
757
760
761
762
763
764
765
768
768
776
784
784
788
792
792
800
808
808
816
816
820
824
826
828
830
832
832
840
848
848
852
856
856
864
872
872
874
878
880
880
884
885
890
896
896
904
912
912
916
920
920
928
936
936
944
944
948
952
954
956
958
960
960
9

7512
7512
7514
7516
7516
7518
7520
7528
7528
7532
7536
7536
7540
7541
7546
7552
7552
7560
7568
7568
7572
7576
7576
7580
7584
7592
7592
7600
7600
7604
7608
7616
7616
7624
7632
7632
7636
7640
7640
7642
7644
7644
7646
7648
7656
7656
7660
7663
7664
7665
7666
7667
7668
7668
7669
7672
7673
7674
7675
7676
7677
7680
7680
7688
7696
7696
7704
7712
7720
7720
7728
7728
7728
7736
7738
7740
7742
7744
7752
7760
7760
7760
7768
7776
7784
7784
7786
7790
7792
7792
7797
7802
7808
7808
7816
7824
7824
7832
7840
7848
7856
7856
7856
7864
7866
7868
7870
7872
7880
7888
7888
7888
7896
7898
7900
7902
7904
7904
7912
7914
7918
7920
7920
7920
7926
7930
7936
7936
7944
7952
7952
7960
7960
7964
7968
7976
7984
7984
7984
7992
8000
8008
8008
8016
8016
8024
8026
8028
8028
8030
8032
8040
8044
8048
8048
8048
8053
8058
8064
8064
8072
8080
8080
8088
8088
8092
8096
8104
8112
8112
8112
8120
8128
8136
8136
8144
8144
8152
8154
8156
8156
8158
8160
8168
8172
8175
8176
8177
8178
8179
8180
8181
8184
8185
8186
8187
8188
8189
8192
8192


In [29]:

print('printing ticks before quantization')
for note in note_list.notes:
    print(note.c_tick)
    
note_list.quantise(min_ppq)

print('printing ticks after quantization')
for note in note_list.notes:
    print(note.c_tick)

printing ticks before quantization
0.0
0.0
480.0
960.0
960.0
1440.0
1920.0
2400.0
2400.0
2880.0
2880.0
3360.0146488845485
3480.018311105686
3600.021973326823
3720.0256355479605
3840.029297769098
4320.043946653646
4800.043946653646
4800.043946653646
5280.043946653646
5280.043946653646
5760.043946653646
6240.043946653646
6240.043946653646
6410.044251838742
6600.054933317058
6720.058595538196
6720.058595538196
7020.067751091039
7200.073244422745
7320.0769066438825
7440.08056886502
7680.0878933072945
7680.0878933072945
8160.0878933072945
8640.087893307295
8640.087893307295
9120.087893307295
9600.087893307295
10080.087893307295
10080.087893307295
10560.087893307295
10560.087893307295
11040.087893307295
11160.091555528432
11280.09521774957
11400.098879970707
11520.102542191844
12000.102542191844
12480.102542191844
12480.102542191844
12960.102542191844
12960.102542191844
13080.106204412981
13200.109866634119
13320.113528855256
13440.117191076393
13920.117191076393
13920.117191076393
14090.117

427200.0292977691
427200.0292977691
427680.0292977691
427800.03295999026
427920.0366222114
428040.0402844325
428160.04394665366
428160.04394665366
428640.04394665366
428880.05127109593
429120.04394665366
429120.04394665366
429120.04394665366
429180.04577776423
429240.0476088748
429300.04943998536
429360.05127109593
429420.0531022065
429600.0585955382
429660.06042664876
429720.06225775933
429780.0640888699
429840.06591998046
429900.06775109103
430080.0
430080.0
430560.0
431040.0
431040.0
431520.0
431520.0
431760.00732444227
432000.01464888453
432480.01464888453
432960.01464888453
432960.01464888453
433200.0219733268
433440.02929776907
433560.03295999026
433680.0366222114
433800.0402844325
433920.04394665366
433920.04394665366
434400.0585955382
434880.0585955382
434880.0585955382
435360.0585955382
435360.0585955382
435600.06591998046
435840.07324442273
436320.07324442273
436490.0735496078
436680.08423108613
436800.08789330727
436800.08789330727
436800.08789330727
437100.09704886016
43740

24480.0
24480.0
24960.0
25440.0
25920.0
25920.0
25920.0
26400.0
26520.0
26640.0
26760.0
26880.0
27360.0
27360.0
27840.0
27840.0
28320.0
28440.0
28560.0
28680.0
28800.0
29280.0
29460.0
29640.0
29760.0
29760.0
30120.0
30360.0
30720.0
30720.0
31200.0
31680.0
31680.0
32160.0
32160.0
32640.0
33120.0
33600.0
33600.0
33840.0
34080.0
34560.0
34560.0
35040.0
35520.0
35520.0
36000.0
36000.0
36120.0
36240.0
36360.0
36480.0
36960.0
37200.0
37440.0
37440.0
37440.0
37740.0
38100.0
38400.0
38400.0
38880.0
39360.0
39360.0
39840.0
39840.0
40320.0
40800.0
41280.0
41280.0
41520.0
41760.0
42240.0
42240.0
42720.0
43200.0
43200.0
43680.0
43680.0
43800.0
43920.0
44040.0
44160.0
44640.0
44880.0
45120.0
45120.0
45120.0
45180.0
45240.0
45300.0
45360.0
45420.0
45600.0
45660.0
45720.0
45780.0
45840.0
45900.0
46080.0
46080.0
46560.0
47040.0
47040.0
47280.0
47520.0
47520.0
48000.0
48480.0
48480.0
48960.0
48960.0
49200.0
49440.0
49560.0
49680.0
49800.0
49920.0
49920.0
50400.0
50880.0
50880.0
51120.0
51360.0
51360.0


464640.0
465120.0
465600.0
465600.0
465600.0
466080.0
466560.0
467040.0
467040.0
467220.0
467400.0
467520.0
467520.0
467820.0
468120.0
468480.0
468480.0
468960.0
469440.0
469440.0
469920.0
470400.0
470880.0
471360.0
471360.0
471360.0
471840.0
471960.0
472080.0
472200.0
472320.0
472800.0
473280.0
473280.0
473280.0
473760.0
473880.0
474000.0
474120.0
474240.0
474240.0
474720.0
474900.0
475080.0
475200.0
475200.0
475200.0
475560.0
475800.0
476160.0
476160.0
476640.0
477120.0
477120.0
477600.0
477600.0
477840.0
478080.0
478560.0
479040.0
479040.0
479040.0
479520.0
480000.0
480480.0
480480.0
480960.0
480960.0
481440.0
481560.0
481680.0
481680.0
481800.0
481920.0
482400.0
482640.0
482880.0
482880.0
482880.0
483180.0
483540.0
483840.0
483840.0
484320.0
484800.0
484800.0
485280.0
485280.0
485520.0
485760.0
486240.0
486720.0
486720.0
486720.0
487200.0
487680.0
488160.0
488160.0
488640.0
488640.0
489120.0
489240.0
489360.0
489360.0
489480.0
489600.0
490080.0
490320.0
490560.0
490560.0
490620.0
4

In [30]:
note_list.simplify_drums()

In [31]:
txt = note_list.return_as_text()

352.0
<class 'float'>


In [32]:
print(txt)

0b101000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b001000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b011000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b001000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 BAR 0b001000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b101000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b011000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b001000000 0b000000000 0b001000000 0b000000000 0b001000000 0b000000000 0b001000000 0b000000000 BAR 0b001000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b001000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b011000000 0b000000000 0b000000

In [33]:
text_file = open("sample.txt", "w")
text_file.write(txt)
text_file.close()

## Testing Txt to MIDI conversion in the following cells

The following cell receives a single line from the txt file until the first 'BAR' element. ```encoded_drums``` is an array where each entry is a note in .txt format, so something like ```0xb011010100```.

```allowed_pitch``` has the following structure ```allowed_pitch = [36, 38, 42, 46, 41, 45, 48, 51, 49]``` where each entry is corresponds to a note in MIDI number (kick, snare ecc..., you can fin the exact notation inside the ```Note_list``` class).

So basically the following function iterates over the single txt note, and for each one of them, iterates over the single binary number and checks if it's equal to one. 
If so, it assigns the equivalent note value taken from ```allowed_pitch```. Basically is doing a 1on1 mapping between the single digit of the binary txt note and the single note taken from ```allowed_pitch```.

In [34]:
#Function that converts txt to notes. The note is represented as a number (in the MIDI scale)

#in encoded drums ho una riga intera dal file (quindi i vari 0xb00101110) 
def text_to_notes(encoded_drums, note_list=None):
    ''' 
    0b0000000000 0b10000000 ...  -> corresponding note. 
    '''
    if note_list == None:
        note_list = Note_List()
#https://www.programiz.com/python-programming/methods/built-in/enumerate enumerate mi ritorna coppie di (indice,valore) 
    for word_idx, word in enumerate(encoded_drums):
        c_tick_here = word_idx*min_ppq 

        for pitch_idx, pitch in enumerate(allowed_pitch):

            if word[pitch_idx+2] == '1':
                new_note = Note(pitch, c_tick_here)
                note_list.add_note(new_note)
    return note_list

The following function uses ```text_to_notes``` in order to fully convert .txt into MIDI. 

It receives as input the file name

In [35]:
import os

def conv_text_to_midi(filename):
    if os.path.exists(filename[:-4]+'.mid'):
        return
    f = open(filename, 'r')
    #These multiple readlines are actually useless. Need to check the output of the NN, but right now they're useless.
    #One single readline is enough
    #f.readline() # title
    #f.readline() # seed sentence
    #legge una riga intera dal file
    sentence = f.readline()
    #splitta gli elementi letti a ogni spazio.
    encoded_drums = sentence.split(' ')

    #find the first BAR

    first_bar_idx = encoded_drums.index('BAR') 

    #encoded_drums = encoded_drums[first_bar_idx:]
    try:
        encoded_drums = [ele for ele in encoded_drums if ele not in ['BAR', 'SONG_BEGIN', 'SONG_END', '']]
    except:
        pdb.set_trace()

    # prepare output
    note_list = Note_List()
    pattern = Containers.Pattern(fmt=0) #Don't know why there's an assertion in the code for fmt=0 if Pattern.len < 1
    track = Containers.Track()
    #??
    PPQ = 480
    min_ppq = PPQ / (event_per_bar/4)
    track.resolution = PPQ # ???? too slow. why??
    pattern.resolution = PPQ
    # track.resolution = 192
    pattern.append(track)

    velocity = 84
    duration = min_ppq*9/10  # it is easier to set new ticks if duration is shorter than _min_ppq

    note_list = text_to_notes(encoded_drums, note_list=note_list)

    max_c_tick = 0 
    not_yet_offed = [] # set of midi.pitch object
    print('entering for note_idx cycle')
    for note_idx, note in enumerate(note_list.notes[:-1]):
        # add onset
        tick_here = note.c_tick - max_c_tick
        pitch_here = pitch_to_midipitch[note.pitch]
        # if pitch_here in cymbals_pitch: # "Lazy-off" for cymbals 
        # 	off = midi.NoteOffEvent(tick=0, pitch=pitch_here)
        # 	track.append(off)

        on = Events.NoteOnEvent(tick=tick_here, velocity=velocity, pitch=pitch_here)
        track.append(on)
        max_c_tick = max(max_c_tick, note.c_tick)
        # add offset for something not cymbal

        # if note_list.notes[note_idx+1].c_tick == note.c_tick:
        # 	if pitch_here not in cymbals_pitch:
        # 	# 	not_yet_offed.append(pitch_here)

        # else:
        # check out some note that not off-ed.
        
        #in questo ciclo pare non ci entri mai. 
        for off_idx, waiting_pitch in enumerate(not_yet_offed):
            print(off_idx)
            if off_idx == 0:
                off = Events.NoteOffEvent(tick=duration, pitch=waiting_pitch)
                max_c_tick = max_c_tick + duration
            else:
                print('appending end note')
                off = Events.NoteOffEvent(tick=0, pitch=waiting_pitch)
            track.append(off)
            not_yet_offed = [] # set of midi.pitch object 

    # finalise
    if note_list.notes == []:
        print ('No notes in %s' % filename)
        return
        pdb.set_trace()
    note = note_list.notes[-1]
    tick_here = note.c_tick - max_c_tick
    pitch_here = pitch_to_midipitch[note.pitch]
    on = Events.NoteOnEvent(tick=tick_here, velocity=velocity, pitch=pitch_here)
    off = Events.NoteOffEvent(tick=duration, pitch=pitch_here)

    for off_idx, waiting_pitch in enumerate(not_yet_offed):
        off = Events.NoteOffEvent(tick=0, pitch=waiting_pitch)

    # end of track event
    eot = Events.EndOfTrackEvent(tick=1)
    track.append(eot)
    # print pattern
    #print(pattern)
    FileIO.write_midifile(filename[:-4]+'.mid', pattern)


In [36]:
conv_text_to_midi("sample.txt")


entering for note_idx cycle


In [31]:
pattern = Containers.Pattern(fmt=0)
print(pattern)

mydy.Pattern(format=0, resolution=220, tracks=\
[])
